### Подготовка. Установка и импорт необходимых библиотек

In [1]:
!pip install py_essentials
!pip install googledrivedownloader
!pip install py_essentials

  Preparing metadata (setup.py) ... done
  Created wheel for py_essentials: filename=py_essentials-1.4.12-py3-none-any.whl size=6915 sha256=49f7f99679d81c903acbe7507b19466e81d94f61abf16203cd28f16d434cfac1
  Stored in directory: /root/.cache/pip/wheels/d6/e0/3e/3e5d30b3f744bacda773940264b12aaa7473bd934f1df1b2f2
Successfully built py_essentials


In [2]:
import tensorflow as tf

from google.colab import drive
from google_drive_downloader import GoogleDriveDownloader as gdd
import os
from pathlib import Path
from PIL import Image, ImageEnhance, ImageOps, ImageDraw, ImageFilter
from py_essentials import hashing as hs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
import seaborn as sns

### Загрузка данных

In [8]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [10]:
BASE_DIR = Path('/content/drive/MyDrive/Skillbox-Computer-vision-thesis/')
BASE_DIR.mkdir(exist_ok=True, parents=True)

TEST_KAGGLE_ZIP = Path('/content/drive/MyDrive/Skillbox-Computer-vision-thesis/test_kaggle')
TRAIN_ZIP = Path('/content/drive/MyDrive/Skillbox-Computer-vision-thesis/train')
PATH_TRAIN = BASE_DIR / 'train'
PATH_TEST = BASE_DIR / 'test_kaggle'
PATH_PROCESSED = BASE_DIR / 'processed'

os.chdir(BASE_DIR)

In [11]:
BASE_DIR = Path('/content/drive/MyDrive/Skillbox-Computer-vision-thesis/')
BASE_DIR.mkdir(exist_ok=True, parents=True)

TEST_KAGGLE_ZIP = BASE_DIR / 'test_kaggle.zip'
TRAIN_ZIP = BASE_DIR / 'train.zip'
PATH_TRAIN = BASE_DIR / 'train'
PATH_TEST = BASE_DIR / 'test_kaggle'
PATH_PROCESSED = BASE_DIR / 'processed'

os.chdir(BASE_DIR)

gdd.download_file_from_google_drive(file_id='15tHAFc2On8-2s0nGAlfAvr9ltZepgtsU',
                                    dest_path=str(TRAIN_ZIP), unzip=False)
gdd.download_file_from_google_drive(file_id='1-4z1xlNBWGtOO6cnQUEM1Jo70Z2Zk3NX',
                                    dest_path=str(TEST_KAGGLE_ZIP), unzip=False)

print(f"Train file exists: {TRAIN_ZIP.exists()}")
print(f"Test file exists: {TEST_KAGGLE_ZIP.exists()}")

if TRAIN_ZIP.exists():
    os.system(f"unzip -q {TRAIN_ZIP}")

if TEST_KAGGLE_ZIP.exists():
    os.system(f"unzip -q {TEST_KAGGLE_ZIP}")

Train file exists: True
Test file exists: True


### Первичный анализ данных

In [12]:
total = 0
print('\033[1m' + 'TRAIN:')
print('\033[0m')
folders = ([name for name in os.listdir(PATH_TRAIN)
            if os.path.isdir(os.path.join(PATH_TRAIN, name))])
for folder in folders:
    contents = os.listdir(os.path.join(PATH_TRAIN,folder))
    print('   Эмоция:',folder,', файлов:',len(contents))
    total += len(contents)

print('\nДиректория TRAIN содержит:',total,'файлов / в',len(os.listdir(PATH_TRAIN)),'категориях.\n')
print('\033[1m' + 'TEST:')
print('\033[0m')
print('Директория TEST содержит:',len(os.listdir(PATH_TEST)),'файлов.')

TRAIN:



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Skillbox-Computer-vision-thesis/train'

In [13]:
idx = 0
size_images = dict()
for folder in folders:
  folder_images = PATH_TRAIN/folder
  for dirpath, _, filenames in os.walk(folder_images):
    for path_image in filenames:
        image = os.path.abspath(os.path.join(dirpath, path_image))
        with Image.open(image) as img:
            weight = os.path.getsize(image)
            width, heigth = img.size
            hash_object = hs.fileChecksum(image, "sha256")
            size_images[idx] = {'weight': weight,
                                'width': width,
                                'heigth': heigth,
                                'emotion' : folder,
                                'filename': path_image,
                                'hash': hash_object}
            idx += 1
df_train = pd.DataFrame(size_images).T
df_train = df_train.astype({'width': int, 'heigth': int})
df_train['notsquare'] = abs(df_train['heigth'] - df_train['width'])

df_train.to_csv(BASE_DIR/'df_train.csv', sep=',', index_label='index')

NameError: name 'folders' is not defined

In [14]:
df_train=pd.read_csv('/content/drive/MyDrive/Skillbox-Computer-vision-thesis/df_train.csv', sep=',')
df_train.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Skillbox-Computer-vision-thesis-Recognizing-human-emotions/df_train.csv'

### Визуализация данных

In [ ]:
sns.histplot(df_train['heigth'])
plt.title('Гистограмма размеров изображений набора TRAIN')

print("Базовые метрики размеров изображений набора TRAIN:")
print(df_train['heigth'].describe())
print("------------------------------------")

print("Самые популярные значения метрики, топ 5")
print(df_train['heigth'].value_counts().nlargest(5))
print("------------------------------------")
print('\nВсе изображения имеют квадратную форму, кроме',df_train[df_train['notsquare'] != 0]['notsquare'].count(),'шт:')
df_train[df_train['notsquare'] != 0].sort_values('notsquare', ascending = False).head()

In [ ]:
image_list = df_train[df_train['notsquare'] != 0].sort_values('notsquare', ascending = False)['filename'].tolist()
emotion_list = df_train[df_train['notsquare'] != 0].sort_values('notsquare', ascending = False)['emotion'].tolist()
plt.figure(figsize=(10,6))

for i in range(6):
    plt.subplot(2,3, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.title(emotion_list[i] + str('/') + image_list[i])
    plt.grid(False)
    image_path = str(PATH_TRAIN) + str('/') + str(emotion_list[i]) + str('/') + str(image_list[i])
    img = mpimg.imread(image_path)
    plt.imshow(img, cmap=plt.cm.gray)

In [ ]:
df_train = pd.read_csv(BASE_DIR/'df_train.csv', sep=',', index_col='index')
df_train_group = df_train.groupby('emotion').agg({'filename': 'count'}).sort_values('filename', ascending = True)
df_train_group['emotion'] = df_train_group.index
ax = df_train_group[['emotion','filename']].plot(kind='bar', title ="Emotion",
                                                      figsize=(10, 5), legend=False, rot = 45,
                                                      fontsize = 10, color = 'lightgreen',
                                                      edgecolor = 'darkblue',linewidth = 1)
ax.set_xlabel("Emotion", fontsize=12)
ax.set_ylabel("Count", fontsize=12)
ax.grid()
plt.show()

In [ ]:
folder_images = PATH_TEST
size_images = dict()

for dirpath, _, filenames in os.walk(folder_images):
    for path_image in filenames:
        image = os.path.abspath(os.path.join(dirpath, path_image))
        with Image.open(image) as img:
            weight = os.path.getsize(image)
            width, heigth = img.size
            size_images[path_image] = {'weight': weight, 'width': width, 'heigth': heigth}

df_test = pd.DataFrame(size_images).T
df_test['notsquare'] = abs(df_test['heigth'] - df_test['width'])
df_test['image_path'] = df_test.index
#сохраним полученный датафрейм в csv-файл
df_test.to_csv(BASE_DIR/'df_test.csv', sep=',', index_label='index')

In [ ]:
df_test=pd.read_csv('/content/drive/MyDrive/Skillbox-Computer-vision-thesis/df_test.csv', sep=',')
df_test.head()

In [ ]:
sns.histplot(df_test['heigth'])
plt.title('Гистограмма размеров изображений тестового набора')

print("Базовые метрики изображений ТЕСТового набора:")
print(df_test['heigth'].describe())
print("------------------------------------")

print("Самые популярные значения метрики, топ 5")
print(df_test['heigth'].value_counts().nlargest(5))
print("------------------------------------")
print('\nВсе изображения имеют квадратную форму, кроме',df_test[df_test['notsquare'] != 0]['notsquare'].count(),'- х:')
print(*df_test[df_test['notsquare'] != 0].index.tolist(),sep=', ',end='\n\n')

### Преобразование данных

In [ ]:
BASE_DIR = Path('/content/drive/MyDrive/Skillbox-Computer-vision-thesis/')
BASE_DIR.mkdir(exist_ok=True, parents=True)
df_train = pd.read_csv(BASE_DIR/'df_train.csv', sep=',', index_col='index')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Skillbox-Computer-vision-thesis-Recognizing-human-emotions/df_train.csv'

In [ ]:
df_clean = df_train[~df_train.duplicated(subset = ['hash'], keep = 'first')].copy()
df_clean = df_clean.groupby('hash').agg({'emotion': 'nunique', 'filename' : 'nunique'})
assert sum(set(df_clean.emotion.tolist())) == 1
assert sum(set(df_clean.filename.tolist())) == 1
print('Дубликаты изображений удалены успешно. Удалено', 50046 - df_clean.shape[0],'записей о файлах.\n')
df_train = df_train.loc[~df_train.duplicated(subset = ['hash'], keep = 'first')]
df_train.reset_index(drop=True, inplace=True)
df_train.drop(['hash'], axis=1, inplace=True)
df_train

### Обработка данных

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import gdown
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
df_train=pd.read_csv('/content/drive/MyDrive/Skillbox-Computer-vision-thesis/df_train.csv', sep=',')
df_test=pd.read_csv('/content/drive/MyDrive/Skillbox-Computer-vision-thesis/df_test.csv', sep=',')

df_train.head()

In [ ]:
df_test.head()

In [ ]:
emotion_labels = {
    'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happy': 4,
    'neutral': 5, 'sad': 6, 'surprise': 7, 'uncertain': 8
}

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = []
        self.labels = []

        for emotion, label in emotion_labels.items():
            emotion_dir = os.path.join(root_dir, emotion)
            for img_file in os.listdir(emotion_dir):
                if img_file.endswith('.jpg'):
                    self.image_files.append(os.path.join(emotion_dir, img_file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = EmotionDataset(root_dir="/content/drive/MyDrive/Skillbox-Computer-vision-thesis/train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
class EmotionClassifier(nn.Module):
    def __init__(self):
        super(EmotionClassifier, self).__init__()
        self.model = models.mobilenet_v2(pretrained=True) # предобученная модель MobileNetV2
        self.model.classifier[1] = nn.Linear(self.model.last_channel, 9)

    def forward(self, x):
        return self.model(x)

### Построение и обучение модели

In [ ]:
model = EmotionClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
num_epochs = 3

for epoch in tqdm(range(num_epochs)):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
torch.save(model.state_dict(), "emotion_classifier_model.pth")

In [ ]:
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.image_files[idx]

In [ ]:
test_dataset = TestDataset(root_dir="/content/drive/MyDrive/Skillbox-Computer-vision-thesiss/test_kaggle", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

### Оценка модели и тестирование

In [ ]:
model.eval()
predictions = []
with torch.no_grad():
    for images, image_files in test_loader:
        images = images.to(device, non_blocking=True)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(zip(image_files, predicted.cpu().numpy()))

In [ ]:
reverse_emotion_labels = {v: k for k, v in emotion_labels.items()}
predictions = [(img, reverse_emotion_labels[pred]) for img, pred in predictions]

### Формирование файла-решения

In [ ]:
submission_df = pd.DataFrame(predictions, columns=['image_path', 'emotion'])
submission_df.to_csv("/content/drive/MyDrive/Skillbox-Computer-vision-thesis/submission.csv", index=False)
print(submission_df.head())